In [1]:
import findspark
import os
findspark.init()
import pyspark
from pyspark import SparkFiles
from pyspark import sql
from pyspark import SparkConf

from pyspark.sql import SQLContext, HiveContext
from pyspark import SparkContext

In [2]:
from pyspark.sql import Row
from pyspark.sql import DataFrameWriter
from pyspark.sql import DataFrameReader
from pyspark.sql import GroupedData

from pyspark import StorageLevel

from pyspark.sql import functions as F
from pyspark.sql.functions import struct
from pyspark.sql import GroupedData

from pyspark.ml.feature import VectorAssembler, VectorIndexer

In [3]:
from functools import partial
import collections
import numpy as np

from datasu.auc import *

from datasu.dicts import *
from datasu.files import *
from datasu.pandas import *
from datasu.persist import *
from datasu.spark import *

import pandas as pd

In [4]:
conf = SparkConf()
conf.set('spark.driver.memory', '12g')
conf.set('spark.python.worker.memory', '2g')
conf.set("spark.driver.maxResultSize", "5g")
conf.set("spark.executor.max", 3)
conf.set('spark.executor.memory', '5g')
conf.set("spark.cores.max", 28)
conf.set('spark.worker.cleanup.enabled', True)
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set('spark.executor.extraJavaOptions', '-XX:+PrintGCDetails -XX:+UseCompressedOops')

conf.setAppName('prepare features')
conf.getAll()

[(u'spark.master', u'spark://spark1.ea.lab:7077'),
 (u'spark.executor.max', u'3'),
 (u'spark.driver.memory', u'12g'),
 (u'spark.submit.pyFiles',
  u'/home/ds/.ivy2/jars/com.databricks_spark-csv_2.10-1.3.0.jar,/home/ds/.ivy2/jars/org.apache.commons_commons-csv-1.1.jar,/home/ds/.ivy2/jars/com.univocity_univocity-parsers-1.5.1.jar'),
 (u'spark.jars',
  u'file:/home/ds/.ivy2/jars/com.databricks_spark-csv_2.10-1.3.0.jar,file:/home/ds/.ivy2/jars/org.apache.commons_commons-csv-1.1.jar,file:/home/ds/.ivy2/jars/com.univocity_univocity-parsers-1.5.1.jar'),
 (u'spark.executor.memory', u'5g'),
 (u'spark.app.name', u'prepare features'),
 (u'spark.driver.maxResultSize', u'5g'),
 (u'spark.files',
  u'file:/home/ds/.ivy2/jars/com.databricks_spark-csv_2.10-1.3.0.jar,file:/home/ds/.ivy2/jars/org.apache.commons_commons-csv-1.1.jar,file:/home/ds/.ivy2/jars/com.univocity_univocity-parsers-1.5.1.jar'),
 (u'spark.serializer', u'org.apache.spark.serializer.KryoSerializer'),
 (u'spark.cores.max', u'28'),
 (u's

In [ ]:
try:
    sc.stop()
except:
    print 'spark context not exists'

In [6]:
   
sc = pyspark.SparkContext(conf=conf)
sqc = pyspark.SQLContext(sc)
# shq = HiveContext(sc)

In [7]:
sc.defaultParallelism, sc.defaultMinPartitions

(2, 2)

In [8]:
csv_reader = sqc.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')

## LOAD DATA

In [9]:
base_data_path = '/home/ds/dev/data/Kagle-ValuesShoppers/'
spark_data_path = 'file://'+ base_data_path + 'spark_data/'
transactions_name = 'transactions'

In [10]:
df_coupons = pd.read_csv(base_data_path+'offers')[['offer','category','company','brand','offervalue','quantity']]
df_offers_ids = pd.read_csv(base_data_path+'trainHistory').rename(columns={'id': 'customer_id'})
df_offers_ids_subm = pd.read_csv(base_data_path+'testHistory').rename(columns={'id': 'customer_id'})
# df_trans_all = pd.read_csv(base_data_path+'transactions_reduced_category').rename(columns={'id': 'customer_id'})

In [11]:
df_offers_hist = pd.merge(df_offers_ids, df_coupons, on=['offer'])
df_offers_hist = df_offers_hist[['customer_id','chain','offer','market','category','company','brand','offerdate','offervalue','quantity','repeattrips','repeater']]

df_offers_subm = pd.merge(df_offers_ids_subm, df_coupons, on=['offer'])
df_offers_subm = df_offers_subm[['customer_id','chain','offer','market','category','company','brand','offerdate','offervalue','quantity']]

In [2]:
ddf_transactions = csv_reader.load(base_data_path+transactions_name, samplingRatio=0.02)
ddf_transactions.rdd.setName(transactions_name)
ddf_transactions.alias('transactions')

ddf_transactions.rdd.getNumPartitions()
ddf_transactions = ddf_transactions.withColumnRenamed('id','customer_id')

NameError: name 'csv_reader' is not defined

In [1]:
ddf_transactions_small, ddf_transactions_big = ddf_transactions.randomSplit([0.2,0.8])

NameError: name 'ddf_transactions' is not defined

In [38]:
ddf_trans = ddf_transactions_small
# ddf_trans.cache()

In [39]:
ddf_trans.count()

349655789

In [16]:
cat_cols = ['chain','market','category','company','brand']
num_cols = ['offervalue','quantity']

## EXPLORE DATA

In [ ]:
ddf_trans.show(5)

In [ ]:
ddf_trans.select('dept').distinct().count()

In [53]:
ddf_trans.select('category').distinct().count()

836

In [ ]:
ddf_trans.select('company').distinct().count()

In [54]:
ddf_trans.select('brand').distinct().count()

35689

In [52]:
ddf_trans.select('customer_id').distinct().count()

311541

## PREPARE DATA

In [ ]:
ddf_offers_hist = sqc.createDataFrame(df_offers_hist)
ddf_offers_subm = sqc.createDataFrame(df_offers_subm)

In [ ]:
ddf_offers_hist_ids = ddf_offers_hist.select('customer_id')
ddf_offers_all_ids = ddf_offers_hist_ids.unionAll(ddf_offers_subm.select('customer_id')).distinct()

ddf_transactions = ddf_transactions.join(ddf_offers_all_ids, on='customer_id', how='leftsemi')

In [ ]:
ddf_transactions = ddf_transactions.repartition(12000)
ddf_transactions.rdd.getNumPartitions()


## PREPARE FEATURES

In [ ]:
summ_grouping = {'total':F.sum, 'average':F.avg }
count_grouping = {'count':F.count }

# count_agg = partial(get_ddf_aggs, agg_columns=['customer_id'], agg_funcs=count_grouping, prefix='agg_')
# total_avg_agg = partial(get_ddf_aggs, agg_columns=['productsize','purchasequantity','purchaseamount'], agg_funcs=summ_grouping, prefix='agg_')

In [ ]:
# grpby_columns = ['customer_id','brand', 'category', 'dept']
grpby_columns = ['customer_id','category']
grpby_columns_name = ['customer_id','brand']


## effective pivot

In [ ]:
import pyspark.sql.types as sql_types
# import pyspark.sql.

from pyspark.mllib.linalg import *
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import VectorUDT, MatrixUDT, Vectors, Matrices

In [40]:
ddf_trans.show(2)

+-----------+-----+----+--------+----------+-----+----------+-----------+--------------+----------------+--------------+
|customer_id|chain|dept|category|   company|brand|      date|productsize|productmeasure|purchasequantity|purchaseamount|
+-----------+-----+----+--------+----------+-----+----------+-----------+--------------+----------------+--------------+
|      86246|  205|   7|     707|1078778070|12564|2012-03-02|       12.0|            OZ|               1|          7.59|
|      86246|  205|  63|    6319| 107654575|17876|2012-03-02|       64.0|            OZ|               1|          1.59|
+-----------+-----+----+--------+----------+-----+----------+-----------+--------------+----------------+--------------+
only showing top 2 rows



In [41]:
total_avg_agg = partial(get_ddf_aggs, agg_columns=['productsize','purchasequantity'], agg_funcs={'total':F.sum}, prefix='agg_', cast_to='double')

In [42]:
ddf_pivot1 = aggregate_and_pivot_into_vector(ddf_trans, id_column='customer_id', 
                                              pivot_column='category', 
                                              aggs=total_avg_agg(['customer_id', 'category']))
                                              

In [43]:
ddf_pivot1.first()

Row(customer_id=98468631, features=SparseVector(1672, {0: 15.0, 1: 23.0, 2: 35.0, 3: 9.0, 4: 57.0, 5: 5.0, 6: 4.0, 7: 37.0, 8: 5.0, 9: 1.0, 10: 2.0, 11: 16.0, 12: 11.0, 15: 7.0, 16: 4.0, 17: 4.0, 18: 5.0, 20: 33.0, 21: 2.0, 22: 6.0, 25: 2.0, 26: 2.0, 28: 3.0, 29: 4.0, 32: 4.0, 33: 4.0, 36: 3.0, 37: 1.0, 40: 13.0, 41: 7.0, 47: 1.0, 48: 9.0, 49: 2.0, 51: 1.0, 52: 1.0, 54: 43.0, 55: 5.0, 58: 8.0, 59: 1.0, 61: 2.0, 62: 1.0, 63: 2.0, 68: 16.0, 71: 2.0, 72: 36.0, 78: 1.0, 79: 1.0, 80: 3.0, 83: 8.0, 84: 1.0, 85: 2.0, 89: 152.0, 90: 17.0, 96: 16.0, 99: 3.0, 100: 6.0, 102: 4.0, 104: 1.0, 105: 2.0, 106: 13.0, 107: 3.0, 109: 17.0, 114: 1.0, 116: 2.0, 118: 4.0, 127: 1.0, 129: 2.0, 130: 1.0, 134: 5.0, 135: 1.0, 137: 2.0, 138: 12.0, 141: 1.0, 142: 4.0, 145: 1.0, 147: 3.0, 148: 2.0, 149: 6.0, 150: 3.0, 152: 5.0, 153: 1.0, 154: 1.0, 156: 3.0, 157: 2.0, 160: 7.0, 162: 6.0, 164: 4.0, 165: 6.0, 166: 1.0, 168: 7.0, 169: 1.0, 176: 3.0, 177: 3.0, 182: 1.0, 183: 1.0, 185: 4.0, 190: 2.0, 191: 3.0, 194: 1.0, 1

In [44]:
total_avg_agg(['customer_id', 'brand'])

[Column<cast((sum(productsize),mode=Complete,isDistinct=false) as double) AS agg_customer_id_brand_productsize_total#180>,
 Column<cast((sum(purchasequantity),mode=Complete,isDistinct=false) as double) AS agg_customer_id_brand_purchasequantity_total#181>]

In [45]:
ddf_pivot2 = aggregate_and_pivot_into_vector(ddf_trans, id_column='customer_id', 
                                              pivot_column='brand', 
                                              aggs=total_avg_agg(['customer_id', 'brand']))                                              

In [46]:
ddf_pivot2.first()

Row(customer_id=98468631, features=SparseVector(71378, {0: 15.0, 1: 14.0, 2: 1.0, 3: 1.0, 5: 3.0, 6: 7.0, 7: 1.0, 9: 3.0, 10: 11.0, 11: 11.0, 12: 2.0, 14: 2.0, 15: 1.0, 17: 1.0, 18: 1.0, 19: 5.0, 22: 5.0, 23: 2.0, 25: 2.0, 27: 4.0, 29: 1.0, 30: 1.0, 31: 18.0, 37: 12.0, 41: 1.0, 42: 4.0, 44: 1.0, 45: 2.0, 46: 10.0, 47: 13.0, 50: 135.0, 52: 7.0, 53: 4.0, 56: 33.0, 60: 5.0, 62: 6.0, 65: 2.0, 66: 1.0, 67: 1.0, 69: 3.0, 70: 4.0, 72: 1.0, 75: 2.0, 79: 4.0, 82: 13.0, 92: 17.0, 97: 2.0, 100: 7.0, 102: 4.0, 103: 3.0, 112: 4.0, 113: 1.0, 122: 1.0, 123: 2.0, 127: 1.0, 128: 3.0, 129: 1.0, 135: 4.0, 136: 4.0, 144: 6.0, 146: 1.0, 147: 2.0, 148: 3.0, 159: 2.0, 167: 1.0, 174: 3.0, 176: 14.0, 182: 2.0, 183: 2.0, 186: 4.0, 187: 1.0, 188: 1.0, 189: 1.0, 190: 2.0, 196: 1.0, 201: 1.0, 209: 2.0, 212: 7.0, 213: 7.0, 214: 6.0, 217: 1.0, 218: 4.0, 222: 15.0, 228: 2.0, 232: 17.0, 235: 1.0, 238: 1.0, 243: 3.0, 247: 2.0, 248: 1.0, 249: 1.0, 261: 7.0, 262: 12.0, 265: 1.0, 266: 4.0, 267: 1.0, 271: 1.0, 273: 3.0, 27

In [47]:
res = merge_features(ddfs=[ddf_pivot1,ddf_pivot2], join_column='customer_id', merge_column='features')

In [48]:
res.first()

Row(customer_id=98468631, features=SparseVector(73050, {0: 15.0, 1: 23.0, 2: 35.0, 3: 9.0, 4: 57.0, 5: 5.0, 6: 4.0, 7: 37.0, 8: 5.0, 9: 1.0, 10: 2.0, 11: 16.0, 12: 11.0, 15: 7.0, 16: 4.0, 17: 4.0, 18: 5.0, 20: 33.0, 21: 2.0, 22: 6.0, 25: 2.0, 26: 2.0, 28: 3.0, 29: 4.0, 32: 4.0, 33: 4.0, 36: 3.0, 37: 1.0, 40: 13.0, 41: 7.0, 47: 1.0, 48: 9.0, 49: 2.0, 51: 1.0, 52: 1.0, 54: 43.0, 55: 5.0, 58: 8.0, 59: 1.0, 61: 2.0, 62: 1.0, 63: 2.0, 68: 16.0, 71: 2.0, 72: 36.0, 78: 1.0, 79: 1.0, 80: 3.0, 83: 8.0, 84: 1.0, 85: 2.0, 89: 152.0, 90: 17.0, 96: 16.0, 99: 3.0, 100: 6.0, 102: 4.0, 104: 1.0, 105: 2.0, 106: 13.0, 107: 3.0, 109: 17.0, 114: 1.0, 116: 2.0, 118: 4.0, 127: 1.0, 129: 2.0, 130: 1.0, 134: 5.0, 135: 1.0, 137: 2.0, 138: 12.0, 141: 1.0, 142: 4.0, 145: 1.0, 147: 3.0, 148: 2.0, 149: 6.0, 150: 3.0, 152: 5.0, 153: 1.0, 154: 1.0, 156: 3.0, 157: 2.0, 160: 7.0, 162: 6.0, 164: 4.0, 165: 6.0, 166: 1.0, 168: 7.0, 169: 1.0, 176: 3.0, 177: 3.0, 182: 1.0, 183: 1.0, 185: 4.0, 190: 2.0, 191: 3.0, 194: 1.0, 

In [51]:
res.count()

311541

In [49]:
write_ddf_to_csv(res, spark_data_path+'ddf_res')

In [50]:
import time
time.ctime()


'Tue Feb 23 20:23:43 2016'

### pivot agg customer_category

In [ ]:
ddf_agg_customer_category = pivot_aggregate(ddf_transactions, grpby_columns=['customer_id','category'],
                                            aggs=count_agg(grpby_columns),
                                            pivot_column='category', pivot_filter_support=None)

In [ ]:
ddf_agg_customer_category.columns

In [ ]:
ddf_agg_customer_category.rdd.setName('ddf_agg_customer_category') \
                         .persist(StorageLevel.MEMORY_AND_DISK_SER)

## merge with offers history

In [ ]:
ddf_agg_customer_category = rename_columns(ddf_agg_customer_category, prefix = 'left', separator='.', columns=cat_cols)

In [ ]:
assemble_columns = list(set(ddf_agg_customer_category.columns) -set(filter_columns('left.*',ddf_agg_customer_category)) \
                    -set(ddf_offers_hist.columns) )+ cat_cols+num_cols

In [ ]:
vecAssembler = VectorAssembler(inputCols=assemble_columns, outputCol="features")

#### train

In [ ]:
ddf_offers__ohagg_cid_category = ddf_offers_hist.join(ddf_agg_customer_category,
                                                     on=['customer_id'], how='left_outer')

In [ ]:
ddf_offers__ohagg_cid_category = vecAssembler.transform(ddf_offers__ohagg_cid_category) \
                                             .select(['customer_id','features', 'repeater']+cat_cols)

In [ ]:
write_ddf_to_csv(ddf_offers__ohagg_cid_category, spark_data_path+'ddf_offers__ohagg_cid_category')

In [ ]:
ddf_offers__ohagg_cid_category.show(3)

#### submission

In [ ]:
ddf_offers_subm__ohagg_cid_category = ddf_offers_subm.join(ddf_agg_customer_category,
                                                      on=['customer_id'], how='left_outer')

In [ ]:
ddf_offers_subm__ohagg_cid_category = vecAssembler.transform(ddf_offers_subm__ohagg_cid_category) \
                                    .select(['customer_id', 'features', F.lit('f').alias('repeater')]+cat_cols)

In [ ]:
write_ddf_to_csv(ddf_offers_subm__ohagg_cid_category, spark_data_path+'ddf_offers_subm__ohagg_cid_category')


#### persist

In [ ]:
def pivot_aggregate(ddf, grpby_columns, pivot_column, aggs, pivot_filter_values=None, pivot_filter_support=None):
    if pivot_filter_support and not pivot_filter_values:        
        frequent = ddf.freqItems([pivot_column], support=pivot_filter_support).first().asDict()[pivot_column+'_freqItems']
        pivot_filter_values = map(str,frequent)
    
    ddf_gr = ddf.groupBy(*grpby_columns)
    ddf_pivot = ddf_gr.pivot(pivot_column, pivot_filter_values)
    ddf_agg = ddf_pivot.agg(*aggs)
    return ddf_agg
    

In [ ]:
def rename_columns(df, prefix='', suffix='', separator='_', columns=None):
    prefix = prefix + separator if prefix else prefix
    suffix = separator + suffix if suffix else suffix
    columns = df.columns if columns is None else columns
    df1 = df.select('*')
    for c in columns:
        df1 = df1.withColumnRenamed(c, prefix + c + suffix)
    return df1       


def filter_columns(expr, df):
    import re
    return filter(lambda c: re.match(expr,c), df.columns)